 # Model Generation
 ## Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
        GradientBoostingClassifier, StackingClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from mlxtend.plotting import plot_learning_curves
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# Hyperparameter tuning functions using keras-tuner
import kerastuner as kt
from tensorboard.plugins.hparams import api as hp
from sklearn.metrics import accuracy_score
import librosa
import sonicboom

from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


 ## Read in the features

In [2]:
#PUT THIS BACK TO TONY'S FILE
#filedata = pd.read_pickle('./output/intermediate-data/filedata-mfcc-100perclass.pkl')

filedata = pd.read_pickle('./output/intermediate-data/filedata-mfccDeltaFeaturesforNN.pkl')


 ### Define convolutional network architecture

In [3]:
#def extract_features(filedata):
#    features, labels = [], []
#    for fn in filedata['path']:
#        sound_clip,sr = librosa.load(fn)
#        melspec = librosa.feature.mfcc(sound_clip, n_mfcc=40)
#        deltas = librosa.feature.delta(melspec)
#        combi = np.dstack((melspec, deltas))
#        features.append(combi)
#    return features


In [4]:
# filedata = filedata.groupby(
#     'class', 
#     as_index=False, 
#     group_keys=False
# ).apply(lambda x: x.sample(10))


In [5]:
#fff = extract_features(filedata)
#filedata['mfccDelta'] = fff

In [6]:
filedata, testdata = train_test_split(filedata, test_size=0.20, random_state=0)

x_test = np.array(testdata['mfccDelta'].tolist())
y_test = np.array(testdata['classID'].tolist())


In [7]:
filedata

,path,slice_file_name,fsID,start,end,salience,fold,classID,class,mfccDelta
5379,data/UrbanSound8K/audio/fold9/70098-3-2-0.wav,70098-3-2-0.wav,70098,8.225276,10.467619,1,9,3,dog_bark,"[[[-514.23914, 32.380196], [-392.34662, 32.380..."
3647,data/UrbanSound8K/audio/fold7/201988-5-0-14.wav,201988-5-0-14.wav,201988,8.671607,12.671607,1,7,5,engine_idling,"[[[-213.20267, -5.8873453], [-225.31009, -5.88..."
1292,data/UrbanSound8K/audio/fold8/80589-0-0-2.wav,80589-0-0-2.wav,80589,1.000000,5.000000,1,8,0,air_conditioner,"[[[-95.60233, -1.3208951], [-106.92463, -1.320..."
3480,data/UrbanSound8K/audio/fold7/102853-8-0-4.wav,102853-8-0-4.wav,102853,2.000000,6.000000,2,7,8,siren,"[[[-217.7957, -0.92076594], [-209.76605, -0.92..."
2247,data/UrbanSound8K/audio/fold10/187110-2-0-8.wav,187110-2-0-8.wav,187110,4.000000,8.000000,1,10,2,children_playing,"[[[-109.84908, -2.8048198], [-121.90108, -2.80..."
...,...,...,...,...,...,...,...,...,...,...
4373,data/UrbanSound8K/audio/fold5/178260-7-5-0.wav,178260-7-5-0.wav,178260,81.308422,84.964376,1,5,7,jackhammer,"[[[-105.01146, 0.5509868], [-80.28012, 0.55098..."
7891,data/UrbanSound8K/audio/fold1/72261-3-0-23.wav,72261-3-0-23.wav,72261,12.973879,16.973879,1,1,3,dog_bark,"[[[-417.51846, -4.621727], [-418.86343, -4.621..."
4859,data/UrbanSound8K/audio/fold5/178260-7-6-0.wav,178260-7-6-0.wav,178260,86.426758,89.351522,1,5,7,jackhammer,"[[[-129.96436, 3.0118656], [-91.63025, 3.01186..."
3264,data/UrbanSound8K/audio/fold2/169098-7-1-0.wav,169098-7-1-0.wav,169098,32.848496,36.848496,2,2,7,jackhammer,"[[[-133.45607, 2.809633], [-138.89018, 2.80963..."


In [8]:
testdata

,path,slice_file_name,fsID,start,end,salience,fold,classID,class,mfccDelta
6532,data/UrbanSound8K/audio/fold4/169466-4-3-7.wav,169466-4-3-7.wav,169466,206.787355,210.787355,2,4,4,drilling,"[[[-84.27396, -1.5654161], [-84.010056, -1.565..."
5750,data/UrbanSound8K/audio/fold9/52411-9-0-52.wav,52411-9-0-52.wav,52411,26.000000,30.000000,2,9,9,street_music,"[[[-307.15015, -0.9330312], [-301.60693, -0.93..."
6685,data/UrbanSound8K/audio/fold4/47926-3-1-0.wav,47926-3-1-0.wav,47926,3.315978,3.752291,1,4,3,dog_bark,"[[[-527.90405, 30.75176], [-476.66122, 30.7517..."
4103,data/UrbanSound8K/audio/fold7/135527-6-10-0.wav,135527-6-10-0.wav,135527,32.860655,33.714179,2,7,6,gun_shot,"[[[-47.724823, 8.265683], [-33.562405, 8.26568..."
8012,data/UrbanSound8K/audio/fold1/143651-2-0-0.wav,143651-2-0-0.wav,143651,0.000000,4.000000,2,1,2,children_playing,"[[[-905.33765, -4.067739e-14], [-905.33765, -4..."
...,...,...,...,...,...,...,...,...,...,...
8154,data/UrbanSound8K/audio/fold1/180937-7-4-2.wav,180937-7-4-2.wav,180937,464.203523,468.203523,2,1,7,jackhammer,"[[[-103.6704, 3.0704038], [-117.2488, 3.070403..."
675,data/UrbanSound8K/audio/fold3/166101-5-2-0.wav,166101-5-2-0.wav,166101,19.712567,23.712567,1,3,5,engine_idling,"[[[-110.27481, -13.6456], [-132.45062, -13.645..."
4519,data/UrbanSound8K/audio/fold5/90013-7-0-5.wav,90013-7-0-5.wav,90013,35.352372,39.352372,1,5,7,jackhammer,"[[[-160.25584, -2.598031], [-173.30704, -2.598..."
768,data/UrbanSound8K/audio/fold3/199769-1-0-10.wav,199769-1-0-10.wav,199769,5.848273,9.848273,1,3,1,car_horn,"[[[-98.55189, -7.7338123], [-124.830864, -7.73..."


In [9]:

def get_cnn():
    num_filters = [24,32,64,128] 
    pool_size = (2, 2) 
    kernel_size = (3, 3)  
    input_shape = (40, 173, 2)
    num_classes = 10
    keras.backend.clear_session()
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(24, kernel_size, input_shape=input_shape,
                padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(32, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(64, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(128, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  

    model.add(keras.layers.GlobalMaxPooling2D())
    model.add(keras.layers.Dense(256, activation="relu"))
    model.add(keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(optimizer=keras.optimizers.Adam(1e-3), 
        loss=keras.losses.SparseCategoricalCrossentropy(), 
        metrics=["accuracy"])
    return model


 ### Train and evaluate via 10-Folds cross-validation

In [10]:
accuracies = []
folds = np.array(list(range(1,11)))
kf = KFold(n_splits=10)
trainaccuracies = []
valaccuracies = []
testaccuracies = []
i = 0
logdir = './logs/2DMFCCwDelta/'
num_epochs = 1000
num_waits = 50
verbosity = 1

for train_index, test_index in kf.split(folds):
    traindata = filedata[filedata['fold'].isin(list(folds[train_index]))]
    x_train = np.array(traindata['mfccDelta'].tolist())
    y_train = np.array(traindata['classID'].tolist())

    testdata = filedata[filedata['fold'] == folds[test_index][0]]
    x_val = np.array(traindata['mfccDelta'].tolist())
    y_val = np.array(traindata['classID'].tolist())

    # Checkpoint to continue models, early stopping and tensorboard
    checkpoint = keras.callbacks.ModelCheckpoint(
        logdir + 'best_%d.h5'%i, 
        monitor='val_loss',
        verbose=verbosity, 
        save_weights_only=True, 
        save_best_only=True
    )
    early = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=num_waits
    )
    tb = keras.callbacks.TensorBoard(log_dir=logdir)
    # callbacks_list = [checkpoint, early, tb]
    callbacks_list = [checkpoint, early]

    model = get_cnn()
    # model.fit(x_train, y_train, epochs = 10, batch_size = 24, verbose = 0)
    history = model.fit(
        x_train, 
        y_train, 
        epochs=num_epochs,
        use_multiprocessing=True, 
        verbose=0,
        callbacks=callbacks_list,
        validation_data=(x_val, y_val)
    )
    trainloss, trainacc = model.evaluate(x_train, y_train, verbose=0)
    valloss, valacc = model.evaluate(x_val, y_val, verbose=0)
    testloss, testacc = model.evaluate(x_test, y_test, verbose=0)
    trainaccuracies.append(trainacc)
    valaccuracies.append(valacc)
    testaccuracies.append(testacc)
    print(f"Fold: {i}")
    print("Train Loss: {0} | Accuracy: {1}".format(trainloss, trainacc))
    print("Val Loss: {0} | Accuracy: {1}".format(valloss, valacc))
    print("Test Loss: {0} | Accuracy: {1}".format(testloss, testacc))
    i += 1

# Out of loop, print average of the results
print("===============================================")
print("FINISHED!")
print(f"Number of Epochs per fold: {num_epochs}")
print("Average Train 10 Folds Accuracy: {0}".format(np.mean(trainaccuracies)))
print("Average Val 10 Folds Accuracy: {0}".format(np.mean(valaccuracies)))
print("Average Test 10 Folds Accuracy: {0}".format(np.mean(testaccuracies)))



Epoch 00001: val_loss improved from inf to 1.22052, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00002: val_loss improved from 1.22052 to 0.53541, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00003: val_loss improved from 0.53541 to 0.47176, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00004: val_loss improved from 0.47176 to 0.35034, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00005: val_loss improved from 0.35034 to 0.33816, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00006: val_loss improved from 0.33816 to 0.23847, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00007: val_loss did not improve from 0.23847

Epoch 00008: val_loss improved from 0.23847 to 0.09215, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00009: val_loss did not improve from 0.09215

Epoch 00010: val_loss improved from 0.09215 to 0.08977, saving model to ./logs/2DMFCCwDelta/best_0.h5

Epoch 00011: val_loss did not improve from 0.08977

Epoch 00012: val_loss d

In [11]:
templist = []
templistFILE = []
for x in range(len(filedata)):
    templist.append(filedata.iloc[x]['mfccDelta'].shape)
    templistFILE.append(filedata.iloc[x]['slice_file_name'])
temp = pd.DataFrame()
temp['shape'] = templist
temp['slice_file_name'] = templistFILE
temp.to_csv('shapes_mfccDelta.csv')
temp.head()

,shape,slice_file_name
0,"(40, 173, 2)",70098-3-2-0.wav
1,"(40, 173, 2)",201988-5-0-14.wav
2,"(40, 173, 2)",80589-0-0-2.wav
3,"(40, 173, 2)",102853-8-0-4.wav
4,"(40, 173, 2)",187110-2-0-8.wav


In [12]:
len(testdata)

663

In [13]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open('trainHistory.json', 'w'))